In [ ]:
# для примера использую библиотеку neuralprophet
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet
import os
import pickle
from tqdm.auto import tqdm
import datetime

In [ ]:
os.getcwd()

In [ ]:
os.chdir('/Users/andrejmironov/Documents/study_for_github/ML/Total')

In [ ]:
airports = pd.read_csv('airports.csv')
fligshts = pd.read_csv('flights.csv')

In [ ]:
fligshts['Date'] = pd.to_datetime(fligshts[['YEAR','MONTH', 'DAY']])


In [ ]:
#объединяю данные, что бы в данных было найменование аэропорта и код аэропорта
#датасет с цифровой расшифровкой аэропортов не нашел, здесь так же удаляю эти данные

fligshts = fligshts.merge(airports, how='right', left_on='ORIGIN_AIRPORT', right_on='IATA_CODE')

In [ ]:
# получаю список аэропортов вылета, для примера взял топ 5 по количеству авиарейсов

ORIGIN_AIRPORT = list(fligshts.groupby('ORIGIN_AIRPORT')['Date'].count().sort_values().tail(5).reset_index()['ORIGIN_AIRPORT'].unique())


In [ ]:
#из списка выше формирую новый датасет
fligshts.query("ORIGIN_AIRPORT in @ORIGIN_AIRPORT", inplace=True)

In [ ]:
#то же для аэропортов отправления
destanation = list(fligshts.DESTINATION_AIRPORT.unique())
destanation = destanation[0:5]
fligshts.query("DESTINATION_AIRPORT in @destanation", inplace=True)

In [ ]:
#выбираю колонки
fligshts = fligshts[['Date', 'ORIGIN_AIRPORT', 'DEPARTURE_DELAY', 'DESTINATION_AIRPORT', 'ARRIVAL_DELAY']].reset_index()
fligshts.dropna(inplace=True)

In [ ]:
#меняю тип данных
fligshts['ARRIVAL_DELAY'] = fligshts['ARRIVAL_DELAY'].astype('int')
fligshts['DEPARTURE_DELAY'] = fligshts['DEPARTURE_DELAY'].astype('int')


In [ ]:
#здесь формирую колонку PROBABILITY - если самолет опоздал, то значение 0
#прилетел раньше или в срок - значение 1


fligshts['PROBABILITY'] = np.where(fligshts['DEPARTURE_DELAY']>0, 0, 1)


In [ ]:
#в данных есть пропуск месяца - октябрь
fligshts = fligshts.loc[fligshts['Date'].between('2015-01-01', '2015-09-01')]

In [ ]:
# создаем каталоги для аэропортов отправления
for airport in ORIGIN_AIRPORT:
    os.mkdir("{}".format(airport))

In [ ]:
# обучаем модель

for airport in tqdm(ORIGIN_AIRPORT):
    rsme = pd.DataFrame() - #далее буду добавлять метрику на обучающей выборке
    df = fligshts.query(f'ORIGIN_AIRPORT == "{airport}"') #датафрейм с конкретным аэропортом прилета
    DESTINATION_AIRPORT = list(df.DESTINATION_AIRPORT.unique()) #список аэропортов прилета для конкретного аэропорта вылета
    DESTINATION_AIRPORT = DESTINATION_AIRPORT[0:5]
    for d_airport in tqdm(DESTINATION_AIRPORT):
        A = df.query(f'DESTINATION_AIRPORT == "{d_airport}"') #фрейм с конкретным аэропортом прилета
        if A.shape[0] > 10:
            B = A.groupby('Date')['PROBABILITY'].mean().reset_index() #фрейм с датой и средним значением вероятности задержки в этот аэропорт по дням
            B.columns = ['ds', 'y'] #привожу колонки к наименованию, которые необходимы для модели
            m = NeuralProphet(weekly_seasonality = True, daily_seasonality = False, seasonality_mode='multiplicative') #создаем экземпляр класса, 
                                                                                                                        #добавляю параметр сезонный шаблон
            m = m.add_country_holidays("US") #добавляю список праздников в стране
            metrix = m.fit(B, freq="D") #обучаем модель
            a = float(metrix.RMSE[-1:]) #из последней эпохи обучения сохраняю значение RSME
            d = {'name':'{}'.format(d_airport), 'rsme':a}# в словарь добавляю аэропорт отправления и соответствующую ему метрику
            rsme = rsme.append(d, ignore_index=True) #добавляю данные в фрейм
            with open('{}'.format(airport)+'/'+'{}'.format(d_airport)+'.pkl', 'wb') as f: # записываем модель
                pickle.dump(m, f)

        with open('{}'.format(airport)+'/'+'metrix'+'.pkl', 'wb') as f: # записываем метрики
            pickle.dump(rsme, f)
  

In [ ]:
# Выборку из flights сохраняю в файл, нужен для второго файла stream.py
fligshts.to_csv('flights_2.csv') 